In [1]:
import h5py
import numpy as np
import os
import glob
import time

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tarfile

In [4]:
SUPPRESS_PLOTS = False

# Helper Functions

In [87]:
from IPython.core.display import HTML
def normalize_rows(data):
    ''' 
    Normalize feature vector to be unit length feature vector
    '''
    num_rows, num_feats = data.shape
    for i in range(num_rows):
        # Take the Frobenius norm of the data  
        data[i, :] = np.linalg.norm(data[i,:], 'fro')
    return data
def get_table_html(image_paths, height ="100px"):
    '''
    This takes in a list of lists of image paths or none
    Input: [[None, path1, None], [path2, path3, path4]]
    Output: An HTML table containing the images
    '''
    if len(image_paths) != 2: 
        raise NotImplementedError('Only 2 inputs supported, not %d'%len(image_paths))
    if len(image_paths[0]) != len(image_paths[1]):
        raise ValueError('Inputs rows must have same length')
    
    row_size = len(image_paths[1])
    output_table = [[],[]]
    for row in range(len(image_paths)):
        for col, path in enumerate(image_paths[row]):
            if path is None:
                output_table[row].append('<td />')
            else:
                output_table[row].append('<td><img src="%s" width="auto" height="%s"/></td>'%(path,height))
    output_html = ""
    output_html += "<table>"
    for row in output_table:
        output_html += "<tr>" + ''.join(row) + "</tr>"
    output_html += "</table>"
    return output_html
    

# Load Data

In [6]:
# Store All Data in Memory
all_data = None
all_fnames = None
for i, fname in enumerate(glob.glob('hdf5/*')[:50]):
    print 'Loading %s'%fname, i
    raw_data = h5py.File(fname, 'r')
    data = normalize_rows(np.array(raw_data['feats']).transpose())
    if all_fnames is not None:
        all_data = np.concatenate((all_data, data))
        all_fnames.extend(raw_data['filenames'][()].tolist())
    else:
        all_data = data
        all_fnames = raw_data['filenames'][()].tolist()

Loading hdf5/183.hdf5 0
Loading hdf5/530.hdf5 1
Loading hdf5/989.hdf5 2
Loading hdf5/993.hdf5 3
Loading hdf5/980.hdf5 4
Loading hdf5/188.hdf5 5
Loading hdf5/191.hdf5 6
Loading hdf5/215.hdf5 7
Loading hdf5/201.hdf5 8
Loading hdf5/985.hdf5 9
Loading hdf5/161.hdf5 10
Loading hdf5/998.hdf5 11
Loading hdf5/996.hdf5 12
Loading hdf5/883.hdf5 13
Loading hdf5/984.hdf5 14
Loading hdf5/586.hdf5 15
Loading hdf5/182.hdf5 16
Loading hdf5/283.hdf5 17
Loading hdf5/991.hdf5 18
Loading hdf5/986.hdf5 19
Loading hdf5/124.hdf5 20
Loading hdf5/162.hdf5 21
Loading hdf5/974.hdf5 22
Loading hdf5/175.hdf5 23
Loading hdf5/543.hdf5 24
Loading hdf5/193.hdf5 25
Loading hdf5/975.hdf5 26
Loading hdf5/164.hdf5 27
Loading hdf5/982.hdf5 28
Loading hdf5/171.hdf5 29
Loading hdf5/178.hdf5 30
Loading hdf5/154.hdf5 31
Loading hdf5/882.hdf5 32
Loading hdf5/990.hdf5 33
Loading hdf5/194.hdf5 34
Loading hdf5/174.hdf5 35
Loading hdf5/453.hdf5 36
Loading hdf5/172.hdf5 37
Loading hdf5/997.hdf5 38
Loading hdf5/983.hdf5 39
Loading hd

In [35]:
# Read metadata for files
import bz2
import os
fname_2_path = {}
for line in bz2.BZ2File('/data/yfcc100m_dataset-1.bz2'):
    try:
        line = line.strip().split('\t')
        path = line[14] # 14 = full static path
        fname = os.path.basename(path)
        fname_2_path[fname] = path
    except:
        print line
        raise

# NearPy

In [7]:
# Store data using NearPy
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
dimension=4096
rbp = RandomBinaryProjections('rbp', 10)
nearpy = Engine(dimension, lshashes=[rbp])
for i in range(all_data.shape[0]):
    nearpy.store_vector(all_data[i, :], all_fnames[i])

In [93]:
# Extract Nearest Neighbors for some fo the data
NUM_NEIGHBORS= 3
WIDTH = 100
HEIGHT = "10px"

html = ''
start_time=time.time()
tables = []

# Builds an HTML Table for each set of images and their nearest neighbor 
for src_idx in range(100,105):
    images = [[None]*NUM_NEIGHBORS, [None]*NUM_NEIGHBORS]
    
    # Get Source Image
    src_fname = all_fnames[src_idx]
    if SUPPRESS_PLOTS is False:
        path = fname_2_path[src_fname]
        images[0][int(NUM_NEIGHBORS/2.0)] = path

    # Get nearest neighbors
    nns = [x[1] for x in nearpy.neighbours(all_data[src_idx, :])][1:NUM_NEIGHBORS+1]
    for i, nn_fname in enumerate(nns):
        if SUPPRESS_PLOTS is False:
            path = fname_2_path[nn_fname]
            images[1][i] = path
    tables.append(get_table_html(images))

#print 'Elapsed Time:', time.time() - start_time
HTML(''.join(tables))

# Annoy

In [ ]:
from annoy import AnnoyIndex
annoy = AnnoyIndex(dimension)
total_count = 0
for i in range(all_data.shape[0]):
    annoy.add_item(total_count, all_data[i,:])
    total_count += 1
annoy.build(10) # 10 trees
annoy.save('test.ann')

In [ ]:
NUM_NEIGHBORS= 3
start_time = time.time()
for src_idx in range(200,210):
    images = [[None]*NUM_NEIGHBORS, [None]*NUM_NEIGHBORS]
    
    # Get Source Image
    src_fname = all_fnames[src_idx]
    if SUPPRESS_PLOTS is False:
        path = fname_2_path[src_fname]
        images[0][int(NUM_NEIGHBORS/2.0)] = path
        
    # Get nearest neighbors
    nns = annoy.get_nns_by_item(src_idx,NUM_NEIGHBORS)
    for i, nn in enumerate(nns):
        # Add to plot
        nn_fname = all_fnames[int(nn)]
        if SUPPRESS_PLOTS is False:
            path = fname_2_path[nn_fname]
            images[1][i] = path
    tables.append(get_table_html(images))

#print 'Elapsed Time:', time.time() - start_time
HTML(''.join(tables))

# MINHEAP Implementation

In [13]:
import heapq
class MinHeap:
    ''' A quick and dirty minheap implementation to test out the brute force search'''
    def __init__(self, max_size):
        self.max_size = max_size
        self.data = []
        self.max_acceptable = None

    def get_max(self):
        return self.data[self.get_max_index()][0]

    def get_max_index(self):
        return self.data.index(max(self.data))

    def insert(self, item, index):
        if len(self.data) < self.max_size:
            self.data.append((item, index))
            self.max_acceptable = self.get_max()
        elif item < self.max_acceptable:
            del self.data[self.get_max_index()]
            self.data.append((item, index))
            self.max_acceptable = self.get_max()
    def get_result(self):
        if self.max_size == 1:
            return self.data[0][1]
        else:
            return [v for (k, v) in self.data]

def get_distance(v1, v2):
    # Euclidean
    return np.sqrt(np.sum(np.square(np.subtract(v1,v2))))

def cosine_similarity(v1, v2):
    # Cosine Similarity (assuming normalized vectors)
    return np.dot(v1, v2)

def row_iteratble(matrix):
    for i in range(matrix.shape[0]):
        yield matrix[i, :]

def get_min_distances_index(source, targets, skip_index, heap_size):
    mh = MinHeap(heap_size)
    for i in range(len(targets)):
        if i != skip_index:
            mh.insert(get_distance(source, targets[i, :]), i)
    return mh.get_result()

def find_nn_index(data, index, num_results):
    source = data[index, :]
    min_index = get_min_distances_index(source, data, index, num_results)
    return min_index

In [ ]:
NUM_NEIGHBORS= 3
start_time = time.time()
for src_idx in range(200,210):
    nns = find_nn_index(all_data, src_idx, NUM_NEIGHBORS)
    
    src_fname = os.path.basename(all_fnames[src_idx])
    if SUPPRESS_PLOTS is False:
        path = fname_2_path[src_fname]
        images[0][int(NUM_NEIGHBORS/2.0)] = path
        
    for i, nn in enumerate(nns):
        nn_fname = os.path.basename(all_fnames[nn])
        if SUPPRESS_PLOTS is False:
            path = fname_2_path[nn_fname]
            images[1][i] = path
    tables.append(get_table_html(images))
    
#print 'Elapsed Time:', time.time() - start_time
HTML(''.join(tables))